In [15]:
import pandas as pd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
import segment_speed_utils

i'd start with 5, checking if the columns you want are already there in this file (saved out at the end of stop_arrivals_to_speed, and is a speeds by trip table for that day)

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [4]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage2b': 'nearest/nearest2_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '${speed_vars.time_min_cutoff}', 'max_trip_minutes': 180, 'max_speed': '${speed_vars.max_speed}', 'min_meters_elapsed': 1609, 'segment_meters': 100

In [16]:
analysis_date = "2022-09-21"

In [17]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [18]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/speedmap/speeds_2022-09-21.parquet'

In [12]:
df = pd.read_parquet(path)

In [14]:
df.columns

Index(['trip_instance_key', 'stop_sequence', 'stop_sequence1',
       'shape_array_key', 'stop_id', 'stop_meters', 'arrival_time',
       'arrival_time_sec', 'subseq_arrival_time_sec', 'subseq_stop_meters',
       'meters_elapsed', 'sec_elapsed', 'speed_mph',
       'schedule_gtfs_dataset_key', 'route_id', 'direction_id',
       'scheduled_service_minutes', 'sched_rt_category', 'time_of_day',
       'shape_id', 'segment_id', 'stop_pair_name', 'stop_pair'],
      dtype='object')

In [19]:
route_info = segment_speed_utils.helpers.import_scheduled_trips(
    analysis_date,
    columns=[
        "gtfs_dataset_key", "trip_instance_key",
        "route_id", "direction_id", "route_short_name"
    ],
    get_pandas=True,
)

In [20]:
route_info

,schedule_gtfs_dataset_key,trip_instance_key,route_id,direction_id,route_short_name
0,f61283c9af01105e40bbb479764db6cd,0199084f96e70abb6d1a5240bf576c83,3402,1.0,Route 11
1,f61283c9af01105e40bbb479764db6cd,f866eee05b33e12524c90179ed51d1c4,3402,1.0,Route 11
2,f61283c9af01105e40bbb479764db6cd,58ea512c9cebd150bab9c6315bd63887,3402,1.0,Route 11
3,f61283c9af01105e40bbb479764db6cd,2ea1981d9efa99c354ebd40ab2dfdfac,3402,1.0,Route 11
4,f61283c9af01105e40bbb479764db6cd,8eb1cadb509b80c64de63de106491c78,3402,1.0,Route 11
...,...,...,...,...,...
132728,a6b98092e6d452a480f2d74d0f7517bb,ddd2ff5d49b491b8189cc0306a6ea65a,Ventura County Line,0.0,None
132729,a6b98092e6d452a480f2d74d0f7517bb,82462eddbe57895423cc3b382178d8c3,Antelope Valley Line,1.0,None
132730,a6b98092e6d452a480f2d74d0f7517bb,ee93c5021913eb3e1d2fe9d5850bfe26,Orange County Line,1.0,None
132731,a6b98092e6d452a480f2d74d0f7517bb,b6863dea5a945896a8e74156209421dc,Riverside Line,1.0,None
